In [1]:
from __future__ import print_function
import math
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim

import matplotlib.pyplot as plt
# import matplotlib
# matplotlib.use('Agg')

T = 20
L = 1000
N = 100
np.random.seed(2)
x = np.empty((N, L), 'int64')
x[:] = np.array(range(L)) + np.random.randint(-4*T, 4*T, N).reshape(N, 1)
data = np.sin(x / 1.0 / T).astype('float64')

In [8]:
data.shape

(100, 1000)

In [5]:
class Sequence(nn.Module):
    def __init__(self):
        super(Sequence, self).__init__()
        self.lstm1 = nn.LSTMCell(1, 51)
        self.lstm2 = nn.LSTMCell(51, 1)

    def forward(self, input, future = 0):
        outputs = []
        h_t = Variable(torch.zeros(input.size(0), 51).double(), requires_grad=False)
        c_t = Variable(torch.zeros(input.size(0), 51).double(), requires_grad=False)
        h_t2 = Variable(torch.zeros(input.size(0), 1).double(), requires_grad=False)
        c_t2 = Variable(torch.zeros(input.size(0), 1).double(), requires_grad=False)

        for i, input_t in enumerate(input.chunk(input.size(1), dim=1)):
            h_t, c_t = self.lstm1(input_t, (h_t, c_t))
            h_t2, c_t2 = self.lstm2(c_t, (h_t2, c_t2))
            outputs += [c_t2]
        for i in range(future):# if we should predict the future
            h_t, c_t = self.lstm1(c_t2, (h_t, c_t))
            h_t2, c_t2 = self.lstm2(c_t, (h_t2, c_t2))
            outputs += [c_t2]
        outputs = torch.stack(outputs, 1).squeeze(2)
        return outputs

In [6]:
input = Variable(torch.from_numpy(data[3:, :-1]), requires_grad=False)
target = Variable(torch.from_numpy(data[3:, 1:]), requires_grad=False)

seq = Sequence()
seq.double() # all parameters and buffers
criterion = nn.MSELoss()
optimizer = optim.LBFGS(seq.parameters()) #?

for i in range(15):
    print('STEP: ', i)
    def closure():
        optimizer.zero_grad()
        out = seq(input)
        loss = criterion(out, target)
        print('loss:', loss.data.numpy()[0])
        loss.backward()
        return loss
    optimizer.step(closure)
    # begin to predict
    future = 1000
    pred = seq(input[:3], future = future)
    y = pred.data.numpy()
    # draw the result
    plt.figure(figsize=(30,10))
    plt.title('Predict future values for time sequences\n(Dashlines are predicted values)', fontsize=30) 
    plt.xlabel('x', fontsize=20)
    plt.ylabel('y', fontsize=20)
    plt.xticks(fontsize=20)
    plt.yticks(fontsize=20)
    def draw(yi, color):
        plt.plot(np.arange(input.size(1)), yi[:input.size(1)], color, linewidth = 2.0)
        plt.plot(np.arange(input.size(1), input.size(1) + future), yi[input.size(1):], color + ':', linewidth = 2.0)
    draw(y[0], 'r')
    draw(y[1], 'g')
    draw(y[2], 'b')
    plt.savefig('predict%d.pdf'%i)
plt.close()

STEP:  0
loss: 1.09054480685
loss: 0.826498405052
loss: 0.742258946371
loss: 0.4669266811
loss: 0.331289496509
loss: 0.236026195159
loss: 0.141259129094
loss: 0.0851530448202
loss: 0.0541042162156
loss: 0.0400308146904
loss: 0.0348454014688
loss: 0.0298759461389
loss: 0.0270200258218
loss: 0.0249405790248
loss: 0.0236001234783
loss: 0.0191040909446
loss: 0.0133801821148
loss: 0.0113662791065
loss: 0.00983567492028
loss: 0.00893714785276
STEP:  1
loss: 0.0085707915367
loss: 0.00803035282722
loss: 0.0071075706583
loss: 0.00923094992072
loss: 0.00576149139876
loss: 0.00500355377519
loss: 0.0213273585187
loss: 0.00430447187568
loss: 0.00387833926829
loss: 0.00434628558957
loss: 0.00298486789118
loss: 0.00266434974742
loss: 0.00258906901548
loss: 0.00245833529464
loss: 0.00244060736007
loss: 0.0023920635679
loss: 0.00233781562643
loss: 0.00224966389398
loss: 0.00196080221474
loss: 0.0345017859766
STEP:  2
loss: 0.00191543395452
loss: 0.00190125225438
loss: 0.00187472642341
loss: 0.001856442

KeyboardInterrupt: 